In [1]:
import pandas as pd
import numpy as np
import nltk
import random
import time
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from copy import copy, deepcopy
import collections
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from pandas.tseries.offsets import *
import re
import collections
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import svm
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from os import path
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from wordcloud import WordCloud, STOPWORDS

porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

/Users/MaxTan/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
data_ads = "/Users/MaxTan/Documents/CU_16fall/BDA/Project/Data/"

user_raw = pd.read_csv(data_ads + "wow3_user2.csv", names = ["user_id", "review_count", "average_stars", 
                                               "friends", "fans", "votes_cool", "votes_funny", 
                                               "votes_useful", "num", "degree", "coefficient"])

business_raw = pd.read_csv(data_ads+"wow3_business_mysql.csv", names = ["business_id", "name", "latitude", 
                                                           "longitude", "stars", "categories", 
                                                           "review_count", "open"])

review_raw = pd.read_csv(data_ads+"wow3_review_mysql.csv", names = ["review_id", "user_id", "business_id", 
                                                       "stars", "text", "date", "votes_funny", 
                                                       "votes_useful"])

all_raw = pd.read_csv(data_ads+"wow3_all2.csv", names = ["business_id", 
                                                       "name", "stars_business","business_review_count",
                                                       "categories", "open", "review_id", 
                                                       "stars_review", "text", "date", "votes_funny", 
                                                       "votes_useful","user_id", "user_review_count", 
                                                       "average_stars","friends", "fans", "num", 
                                                       "degree", "coefficient"])


/Users/MaxTan/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#2.Topic Model and Sentimental Analysis by Using Key Word:
search_word = 'pizza'

In [4]:
#The following codes take 20 minutes to generate a new csv file, which I've store it in the same folder:
"""
bus_gb_df = deepcopy(all_raw)
#Group by different businesses for different rates:
bus_df = bus_gb_df.groupby(['business_id']).mean()
bus_df['business_id'] = bus_df.index
bus_df.index = range(len(bus_df))
bus = list(set(bus_df['business_id']))
ind = [True if i in bus else False for i in business_raw['business_id']]
business_revised = business_raw[ind]

#Merge two tables:
lda_df = pd.merge(bus_df,business_revised,on='business_id')

start_time = time.time()
bus_topic = dict()

for i in range(len(all_raw)):
    ly = all_raw.text[i].lower()
    ly = ly.decode('utf-8')
    
    #1.Tokenization:
    tokens = nltk.word_tokenize(ly)
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue

    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")]

    tokens = word_stop
    
    
    temp_bus = all_raw.business_id[i]
    if temp_bus not in bus_topic:
        bus_topic[temp_bus] = tokens
    else:
        bus_topic[temp_bus] += tokens



print("--- %s seconds ---" % (time.time() - start_time))

lda_df['tokens'] = pd.Series(['']*len(lda_df),index = lda_df.index)
print len(bus_topic)
for bus in bus_topic:
    #print bus
    tempdf = lda_df[lda_df['business_id']==bus]
    temptext = tempdf['categories'].values[0].lower()[1:-1]+', '.join(bus_topic[bus])
    lda_df.set_value([tempdf.index[0]],'tokens',temptext)

lda_df.to_csv('business_LDA.csv')"""

'\nbus_gb_df = deepcopy(all_raw)\n#Group by different businesses for different rates:\nbus_df = bus_gb_df.groupby([\'business_id\']).mean()\nbus_df[\'business_id\'] = bus_df.index\nbus_df.index = range(len(bus_df))\nbus = list(set(bus_df[\'business_id\']))\nind = [True if i in bus else False for i in business_raw[\'business_id\']]\nbusiness_revised = business_raw[ind]\n\n#Merge two tables:\nlda_df = pd.merge(bus_df,business_revised,on=\'business_id\')\n\nstart_time = time.time()\nbus_topic = dict()\n\nfor i in range(len(all_raw)):\n    ly = all_raw.text[i].lower()\n    ly = ly.decode(\'utf-8\')\n    \n    #1.Tokenization:\n    tokens = nltk.word_tokenize(ly)\n    #2.Special Characters Removed:\n    tokens = [re.sub(\'[^A-Za-z0-9]+\', \'\', mystring) for mystring in tokens]\n    tokens_lem_stem = []\n    for word in tokens:\n        try:\n            #3.First Lemmatization, 4.then Stemming:\n            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]\n      

In [5]:
# Create Tokenizer:
tokenizer = RegexpTokenizer(r'\w+')
# Create English stop words list
en_stop = get_stop_words('en')
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# Load CSV file directly:
lda_df2 = pd.read_csv('business_LDA.csv')

In [6]:
"""
#1. Sort by stars:
ind = [True if search_word in token else False for token in lda_df2.tokens.values]
lda_df_sub = lda_df2[ind]
lda_df_target = lda_df_sub.sort(['stars_review'],ascending=[0]).iloc[:6]
doc_set = list(lda_df_target.tokens.values)
"""
#2. Sort by number of key word:
lda_df2['count_sw'] = pd.Series([0]*len(lda_df2),index = lda_df2.index)
for i in range(len(lda_df2)):
    temp_value = collections.Counter([item.strip() for item in lda_df2.tokens[i].split(",")])[search_word]
    lda_df2.set_value(i,'count_sw',temp_value)
lda_df_target = lda_df2.sort(['count_sw'],ascending=[0]).iloc[:3]
doc_set = list(lda_df_target.tokens.values)

/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [7]:
for i in range(len(doc_set)):
    
    start_time = time.time()
    
    texts = []
    # clean and tokenize document string
    raw = doc_set[i].lower()
    #1.Tokenization:
    tokens = tokenizer.tokenize(raw)
    #2.Special Characters Removed:
    tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
    tokens_lem_stem = []
    for word in tokens:
        try:
            #3.First Lemmatization, 
            #4.then Stemming:
            tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
        except UnicodeDecodeError:
            continue
    #5.Remove Punctuation:
    word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation] 
    #6.Remove stop words:
    word_stop = [w for w in word_punct if w not in stopwords.words("english")] 
    # add tokens to list
    texts.append(word_stop)
    # turn our tokenized documents into a id <-> term dictionary
    dictionary = corpora.Dictionary(texts)
    # convert tokenized documents into a document-term matrix
    corpus = [dictionary.doc2bow(text) for text in texts]
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=50)
    result = ldamodel.print_topics(num_topics=10, num_words=10)
    result1 = [j[1] for j in result]
    res_topic = []
    for item in result1:  
        res_topic+=str(item).split('"')[1::2]
    text = " ".join(res_topic)
    top_words = [item[0] for item in collections.Counter(text.split()).most_common(10)]
    # Generate a word cloud image
    wordcloud = WordCloud().generate(text)
    # Display the generated image:
    # the matplotlib way:
    plt.figure(figsize=(100,100),dpi =300)
    plt.imshow(wordcloud)
    plt.axis("off")
    
    # take relative word frequencies into account, lower max_font_size
    wordcloud = WordCloud(max_font_size=80,stopwords=STOPWORDS,font_path = 'EraserRegular.ttf',
                          background_color='blue',max_words=100,width=600,height=600).generate(text)
    plt.figure()
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(lda_df_target.iloc[i:i+1]['name'].values[0])
    plt.savefig(lda_df_target.iloc[i:i+1]['name'].values[0]+".jpg",figsize=(100,100),dpi=100)
    plt.close()
    
    
    #Sentimental Analysis by Random Foreast:
    
    temp_biz = lda_df_target.iloc[i:i+1]['business_id'].values[0]
    temp_rev = review_raw[review_raw['business_id']==temp_biz]

    traindf = temp_rev
    traindf['sentiment'] = pd.Series([0]*len(traindf),index = traindf.index)
    for i in traindf.index:
        star = traindf.loc[i]['stars']
        if star >= 4 or star == '5' or star == '4':
            traindf.set_value(i, 'sentiment', 1)

    traindf['review_text'] = pd.Series(traindf.text,index = traindf.index)

    # Unigram Set:
    start_time  = time.time()
    fdist = nltk.FreqDist()
    for i in traindf.index:
        ly = traindf.text[i].lower()
        ly = ly.decode('utf-8')
        #1.Tokenization:
        try:
            tokens = nltk.word_tokenize(ly)
        except UnicodeDecodeError:
            continue

        #2.Special Characters Removed:
        tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
        tokens_lem_stem = []
        for word in tokens:
            try:
                #3.First Lemmatization, 4.then Stemming:
                tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
            except UnicodeDecodeError:
                continue

        #5.Remove Punctuation:
        word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
        #6.Remove stop words:
        word_stop = [w for w in word_punct if w not in stopwords.words("english")]
        fdist = fdist.__add__(nltk.FreqDist(word_stop))
        
    fdist_uni = fdist
    num_unigram = len(fdist)
    # create features for training data
    feat_uni = [i for [i,j] in fdist_uni.most_common(num_unigram)]
    for unigram in feat_uni:
        traindf[unigram] = pd.Series([0.0]*len(traindf),index = traindf.index)
        
    #Try to use unigram to analyze and obtain the feature:
    fdist = nltk.FreqDist()
    for i in traindf.index:
        ly = traindf.review_text[i].lower()
        ly = ly.decode('utf-8')
        #1.Tokenization:
        try:
            tokens = nltk.word_tokenize(ly)
        except UnicodeDecodeError:
            print ly
            continue
        #2.Special Characters Removed:
        tokens = [re.sub('[^A-Za-z0-9]+', '', mystring) for mystring in tokens]
        tokens_lem_stem = []
        for word in tokens:
            try:
                #3.First Lemmatization, 4.then Stemming:
                tokens_lem_stem += [porter_stemmer.stem(wordnet_lemmatizer.lemmatize(word))]
            except UnicodeDecodeError:
                continue
        #5.Remove Punctuation:
        word_punct = [w.lower() for w in tokens_lem_stem if w not in string.punctuation]
        #6.Remove stop words:
        word_stop = [w for w in word_punct if w not in stopwords.words("english")]

        #compute frequency distribution for all unigrams in one review:
        fdist = nltk.FreqDist(word_stop)
        for unigram in feat_uni:
            traindf.set_value(i, unigram, fdist[unigram])
            #if unigram in word_stop:
            #    traindf.set_value(i, unigram, 1)

    clf = RandomForestClassifier(n_estimators= 100,criterion="entropy",oob_score=True)#,min_samples_leaf=15)
    clf = clf.fit(traindf[feat_uni],traindf['sentiment'])

    print 'Correct rate of training set:',clf.score(traindf[feat_uni],traindf['sentiment'])
    print 'Correct oob rate of training set:',clf.oob_score_


    for word in top_words:
        testdata = [0 if item != word else 1 for item in feat_uni]
        print word,clf.predict(testdata)

    print("--- %s seconds ---" % (time.time() - start_time))

/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MaxTan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

Correct rate of training set: 1.0
Correct oob rate of training set: 0.847402597403
great [1]
good [1]
wa [1]
beer [1]
veri [1]
nt [1]
pizza [1]
place [1]
thi [1]
order [1]
--- 394.412029028 seconds ---
Correct rate of training set: 1.0
Correct oob rate of training set: 1.0
wa [1]
mineo [1]
chee [1]
place [1]
thi [1]
nt [1]
pizza [1]
good [1]
like [1]
pittsburgh [1]
--- 347.601261854 seconds ---
Correct rate of training set: 1.0
Correct oob rate of training set: 1.0
wa [1]
gelato [1]
thi [1]
flavor [1]
nt [1]
pizza [1]
good [1]
like [1]
place [1]
mercurio [1]
--- 379.595090866 seconds ---
